In [10]:
import os
import pandas as pd
import numpy as np
import urllib
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score

In [11]:
path = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC'
os.chdir(path)

csv = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\csv.csv'
f = open(csv,'w+')

xl_file = r"C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\colocation_metadata.xlsx"

normtabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\tmean_regression\tables'
maxtabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\maximums\tmax_regression\tables'
mintabpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\minimums\tmin_regression\tables'

normfigpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\normals\tmean_regression\figures'
maxfigpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\maximums\tmax_regression\figures'
minfigpath = r'C:\Users\student\Documents\Alistair CoAgMET Projects\Pythonstuff\QC\minimums\tmin_regression\figures'

In [12]:
xl = pd.ExcelFile(xl_file)

meta = xl.parse('metadata')
meta['sdate'] = meta.sdate.dt.date
meta['edate'] = meta.edate.dt.date
meta = meta.drop(meta.columns[3],axis=1)

In [16]:
sns.set_style('dark')
sns.set(rc={'figure.figsize' : (11.7,8.27)})
n = 0
for index, rows in meta.iterrows():
    try:
        print('Grabbing temperature data from ' + str(meta.ID[n]))
        urllib.request.urlretrieve(
            'http://coagmet.colostate.edu/cgi-bin/web_services.pl?type=daily&sids=' + str(meta.ID[n]) + '&sdate=' + str(meta.sdate[n]) + '&edate=' + str(meta.edate[n]) + '&elems=tave,tmax,tmin',
            filename=csv)
        df = pd.read_csv(csv)
        df = df.reset_index()
    except Exception as e:
        print('Something went wrong while grabbing temperature data from ' + str(meta.ID[n]) + ': ' + e.args[0])
    
    try:
        print('Cleaning temperature data from ' + str(meta.ID[n]))
        df.columns.values[0] = 'station'
        df.columns.values[1] = 'date'
        df.columns.values[2] = 'tave'
        df.columns.values[3] = 'tmax'
        df.columns.values[4] = 'tmin'
        df['date'] = pd.to_datetime(df.date)
        df['julian'] = df.date.dt.strftime('%j')
        
        df.tmax = df.tmax.astype(float)
        df.tmin = df.tmin.astype(float)
        
        df.tmax = 32 + (df.tmax * (9/5))
        df.tmin = 32 + (df.tmin * (9/5))
        
        df.tave = (df.tmax + df.tmin) / 2
        df.tave = df.tave.astype(float)
        
        df = df.replace('M', np.nan, regex=True)
    except Exception as e:
        print('Something went wrong while cleaning temperature data from ' + str(meta.ID[n]) + ': ' + e.args[0])
        
    try:
        print('Calculating daily temperature normals using data from ' + str(meta.ID[n]))
        norms = df.groupby(['julian']).mean().reset_index()
        norms = norms[:-1]
        norms.julian = norms.julian.astype(float)
        norms.tave = norms.tave.astype(float)
    except Exception as e:
        print('Something went wrong while daily calculating temperature normals ' + str(meta.ID[n]) + ': ' + e.args[0])
    
    try:
        print('Creating a smoothing spline from daily average temperature normals at ' + str(meta.ID[n]))
        julian = norms.julian.values
        tave = norms.tave.values
        s = UnivariateSpline(julian,tave,k=5,s=1500)
        ys = s(julian)
        score = r2_score(tave,ys)
        print('The coefficient of determination for the smoothing spline created from ' + str(meta.ID[n]) + ' tave data was: ' + str(score))
    except Exception as e:
        print('Something went wrong while creating the smoothing spline for ' + str(meta.ID[n]) + ': ' + e.args[0])
        
    try:
        print('Fitting tave normal values created by the smoothing spline to a julian day dataframe for ' + str(meta.ID[n]))
        j_day = norms['julian']
        j_day = j_day.reset_index()
        series = pd.Series(ys)
        df2 = series.reset_index()
        smoothed_norms = df2.merge(j_day)
        smoothed_norms.columns.values[1] = 'tave'
    except Exception as e:
        print('Something went wrong while fitting the tave spline values to a julian day dataframe for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Storing the smoothed tave normals for ' + str(meta.ID[n]) + ' in a csv file')
        smoothed_norms.to_csv(normtabpath + r'\tave_smoothed_norms_' + str(meta.ID[n]) + '.csv')
    except Exception as e:
        print('Something went wrong while storing the smoothed tave normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Creating a smoothing spline for daily maximum temperature normals at ' + str(meta.ID[n]))
        tmax = norms.tmax.values
        s2 = UnivariateSpline(julian,tmax,k=5,s=1500)
        ys2 = s2(julian)
        score2 = r2_score(tmax,ys2)
        print('The coefficient of determination for the smoothing spline created from ' + str(meta.ID[n]) + ' tmax data was: ' + str(score2))
    except Exception as e:
        print('Something went wrong while creating a tmax smoothing spline for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Fitting tmax normal values created by the smoothing spline to a julian day dataframe for ' + str(meta.ID[n]))
        series2 = pd.Series(ys2)
        df3 = series2.reset_index()
        smoothed_norms2 = df3.merge(j_day)
        smoothed_norms2.columns.values[1] = 'tmax'
    except Exception as e:
        print('Something went wrong while fitting the tmax spline values to a julian day dataframe for ' +str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Storing the smoothed tmax normals for ' + str(meta.ID[n]) + ' in a csv file')
        smoothed_norms2.to_csv(maxtabpath + r'\tmax_smoothed_norms_' + str(meta.ID[n] + '.csv'))
    except Exception as e:
        print('Something went wrong while storing the smoothed tmax normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Creating a smoothing spline for daily minimum temperature normals at ' + str(meta.ID[n]))
        tmin = norms.tmin.values
        s3 = UnivariateSpline(julian,tmin,k=5,s=1500)
        ys3 = s3(julian)
        score3 = r2_score(tmin,ys3)
        print('The coefficient of determination for the smooth spline created from ' + str(meta.ID[n]) + ' tmin data was: ' + str(score3))
    except Exception as e:
        print('Something went wrong while creating a tmin smoothing spline for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Fitting tmin normal values created by the smoothing spline to a julian day dataframe for ' + str(meta.ID[n]))
        series3 = pd.Series(ys3)
        df4 = series3.reset_index()
        smoothed_norms3 = df4.merge(j_day)
        smoothed_norms3.columns.values[1] = 'tmin'
    except Exception as e:
        print('Something went wrong while fitting the tmin spline values to a julian day dataframe for ' + str(meta.ID[n]))
    try:
        print('Storing the smoothed tmin normals for ' + str(meta.ID[n]) + 'in a csv file')
        smoothed_norms3.to_csv(mintabpath + r'\tmin_smoothed_norms_' + str(meta.ID[n]) + '.csv')
    except Exception as e:
        print('Something went wrong while storing the smoothed tmin normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Creating and storing a figure comparing smoothed tave normals to observed tave normals for ' + str(meta.ID[n]))
        ax = sns.lineplot(julian, tave,label='normal')
        ax.set_title(str(meta.ID[n]) + ' - Average Daily Temperature Normals')
        plt.ylabel('Average Daily Temperature (°F)')
        plt.xlabel('Julian Day')
        plt.plot(julian,ys,'r--',label = 'smoothed normal')
        plt.legend()
        plt.savefig(normfigpath + r'\tave_norm_smoothed_' + str(meta.ID[n]) + '.png')
        plt.clf()
    except Exception as e:
        print('Something went wrong while creating and storing a figure comparing smoothed tave normals to observed tave normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Creating and storing a figure comparing smoothed tmax normals to observed tmax normals for ' + str(meta.ID[n]))
        ax2 = sns.lineplot(julian, tmax, label='normal')
        ax2.set_title(str(meta.ID[n]) + ' - Maximum Daily Temperature Normals')
        plt.ylabel('Maximum Daily Temperature (°F)')
        plt.xlabel('Julian Day')
        plt.plot(julian,ys2,'r--',label = 'smoothed normal')
        plt.legend()
        plt.savefig(maxfigpath + r'\tmax_norm_smoothed_' + str(meta.ID[n]) + '.png')
        plt.clf()
    except Exception as e:
        print('Something went wrong while creating and storing a figure comparing smoothed tmax normals to observed tmax normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    try:
        print('Creating and storing a figure comparing smoothed tmin normals to observed tmin normals for ' + str(meta.ID[n]))
        ax3 = sns.lineplot(julian,tmin, label='normal')
        ax3.set_title(str(meta.ID[n]) + ' - Minimum Daily Temperature Normals')
        plt.ylabel('Minimum Daily Temperature (°F)')
        plt.xlabel('Julian Day')
        plt.plot(julian,ys3,'r--',label='smoothed normal')
        plt.legend()
        plt.savefig(minfigpath + r'\tmin_norm_smoothed_' + str(meta.ID[n]) + '.png')
        plt.clf()
    except Exception as e:
        print('Something went wrong while creating and storing a figure comparing smoothed tmin normals to observed tmin normals for ' + str(meta.ID[n]) + ': ' + e.args[0])
    n=n+1
print('Smoothed normals successfully created!')

Grabbing temperature data from AKR02
Cleaning temperature data from AKR02
Calculating daily temperature normals using data from AKR02
Creating a smoothing spline from daily average temperature normals at AKR02
The coefficient of determination for the smoothing spline created from AKR02 tave data was: 0.9851759102433719
Fitting tave normal values created by the smoothing spline to a julian day dataframe for AKR02
Storing the smoothed tave normals for AKR02 in a csv file
Creating a smoothing spline for daily maximum temperature normals at AKR02
The coefficient of determination for the smoothing spline created from AKR02 tmax data was: 0.9861699563652206
Fitting tmax normal values created by the smoothing spline to a julian day dataframe for AKR02
Storing the smoothed tmax normals for AKR02 in a csv file
Creating a smoothing spline for daily minimum temperature normals at AKR02
The coefficient of determination for the smooth spline created from AKR02 tmin data was: 0.9845897396712675
Fitt

Creating and storing a figure comparing smoothed tmax normals to observed tmax normals for BRK01
Creating and storing a figure comparing smoothed tmin normals to observed tmin normals for BRK01
Grabbing temperature data from BRL02
Cleaning temperature data from BRL02
Calculating daily temperature normals using data from BRL02
Creating a smoothing spline from daily average temperature normals at BRL02
The coefficient of determination for the smoothing spline created from BRL02 tave data was: 0.984431654900317
Fitting tave normal values created by the smoothing spline to a julian day dataframe for BRL02
Storing the smoothed tave normals for BRL02 in a csv file
Creating a smoothing spline for daily maximum temperature normals at BRL02
The coefficient of determination for the smoothing spline created from BRL02 tmax data was: 0.9853009530094164
Fitting tmax normal values created by the smoothing spline to a julian day dataframe for BRL02
Storing the smoothed tmax normals for BRL02 in a csv

Creating and storing a figure comparing smoothed tmax normals to observed tmax normals for COW01
Creating and storing a figure comparing smoothed tmin normals to observed tmin normals for COW01
Grabbing temperature data from CTR01
Cleaning temperature data from CTR01
Calculating daily temperature normals using data from CTR01
Creating a smoothing spline from daily average temperature normals at CTR01
The coefficient of determination for the smoothing spline created from CTR01 tave data was: 0.9910054645707336
Fitting tave normal values created by the smoothing spline to a julian day dataframe for CTR01
Storing the smoothed tave normals for CTR01 in a csv file
Creating a smoothing spline for daily maximum temperature normals at CTR01
The coefficient of determination for the smoothing spline created from CTR01 tmax data was: 0.9874743757461921
Fitting tmax normal values created by the smoothing spline to a julian day dataframe for CTR01
Storing the smoothed tmax normals for CTR01 in a cs

Creating and storing a figure comparing smoothed tave normals to observed tave normals for EAC01
Creating and storing a figure comparing smoothed tmax normals to observed tmax normals for EAC01
Creating and storing a figure comparing smoothed tmin normals to observed tmin normals for EAC01
Grabbing temperature data from EKT01
Cleaning temperature data from EKT01
Calculating daily temperature normals using data from EKT01
Creating a smoothing spline from daily average temperature normals at EKT01
The coefficient of determination for the smoothing spline created from EKT01 tave data was: 0.9830121621380417
Fitting tave normal values created by the smoothing spline to a julian day dataframe for EKT01
Storing the smoothed tave normals for EKT01 in a csv file
Creating a smoothing spline for daily maximum temperature normals at EKT01
The coefficient of determination for the smoothing spline created from EKT01 tmax data was: 0.9867526531722889
Fitting tmax normal values created by the smoothi

Creating and storing a figure comparing smoothed tmax normals to observed tmax normals for FWL01
Creating and storing a figure comparing smoothed tmin normals to observed tmin normals for FWL01
Grabbing temperature data from GLY04
Cleaning temperature data from GLY04
Calculating daily temperature normals using data from GLY04
Creating a smoothing spline from daily average temperature normals at GLY04
The coefficient of determination for the smoothing spline created from GLY04 tave data was: 0.9852058537200429
Fitting tave normal values created by the smoothing spline to a julian day dataframe for GLY04
Storing the smoothed tave normals for GLY04 in a csv file
Creating a smoothing spline for daily maximum temperature normals at GLY04
The coefficient of determination for the smoothing spline created from GLY04 tmax data was: 0.9860724430975776
Fitting tmax normal values created by the smoothing spline to a julian day dataframe for GLY04
Storing the smoothed tmax normals for GLY04 in a cs

...

Creating and storing a figure comparing smoothed tmax normals to observed tmax normals for LSL01
Creating and storing a figure comparing smoothed tmin normals to observed tmin normals for LSL01
Grabbing temperature data from MCL01
Cleaning temperature data from MCL01
Calculating daily temperature normals using data from MCL01
Creating a smoothing spline from daily average temperature normals at MCL01
The coefficient of determination for the smoothing spline created from MCL01 tave data was: 0.9858797303435279
Fitting tave normal values created by the smoothing spline to a julian day dataframe for MCL01
Storing the smoothed tave normals for MCL01 in a csv file
Creating a smoothing spline for daily maximum temperature normals at MCL01
The coefficient of determination for the smoothing spline created from MCL01 tmax data was: 0.9856290541499008
Fitting tmax normal values created by the smoothing spline to a julian day dataframe for MCL01
Storing the smoothed tmax normals for MCL01 in a cs

Creating and storing a figure comparing smoothed tmax normals to observed tmax normals for PAI01
Creating and storing a figure comparing smoothed tmin normals to observed tmin normals for PAI01
Grabbing temperature data from PKH01
Cleaning temperature data from PKH01
Calculating daily temperature normals using data from PKH01
Creating a smoothing spline from daily average temperature normals at PKH01
The coefficient of determination for the smoothing spline created from PKH01 tave data was: 0.9883761842357521
Fitting tave normal values created by the smoothing spline to a julian day dataframe for PKH01
Storing the smoothed tave normals for PKH01 in a csv file
Creating a smoothing spline for daily maximum temperature normals at PKH01
The coefficient of determination for the smoothing spline created from PKH01 tmax data was: 0.9864811543111517
Fitting tmax normal values created by the smoothing spline to a julian day dataframe for PKH01
Storing the smoothed tmax normals for PKH01 in a cs

Creating and storing a figure comparing smoothed tmax normals to observed tmax normals for SBT01
Creating and storing a figure comparing smoothed tmin normals to observed tmin normals for SBT01
Grabbing temperature data from SCM01
Cleaning temperature data from SCM01
Calculating daily temperature normals using data from SCM01
Creating a smoothing spline from daily average temperature normals at SCM01
The coefficient of determination for the smoothing spline created from SCM01 tave data was: 0.9872575639615374
Fitting tave normal values created by the smoothing spline to a julian day dataframe for SCM01
Storing the smoothed tave normals for SCM01 in a csv file
Creating a smoothing spline for daily maximum temperature normals at SCM01
The coefficient of determination for the smoothing spline created from SCM01 tmax data was: 0.9862702242814267
Fitting tmax normal values created by the smoothing spline to a julian day dataframe for SCM01
Storing the smoothed tmax normals for SCM01 in a cs

Creating and storing a figure comparing smoothed tmax normals to observed tmax normals for WCF01
Creating and storing a figure comparing smoothed tmin normals to observed tmin normals for WCF01
Grabbing temperature data from WFD01
Cleaning temperature data from WFD01
Calculating daily temperature normals using data from WFD01
Creating a smoothing spline from daily average temperature normals at WFD01
The coefficient of determination for the smoothing spline created from WFD01 tave data was: 0.9867833330018705
Fitting tave normal values created by the smoothing spline to a julian day dataframe for WFD01
Storing the smoothed tave normals for WFD01 in a csv file
Creating a smoothing spline for daily maximum temperature normals at WFD01
The coefficient of determination for the smoothing spline created from WFD01 tmax data was: 0.9888612470008472
Fitting tmax normal values created by the smoothing spline to a julian day dataframe for WFD01
Storing the smoothed tmax normals for WFD01 in a cs

<Figure size 842.4x595.44 with 0 Axes>